In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from libs.signals import signals

import hvplot.pandas
import numpy as np

In [ ]:
def technical_indicator_signal(asset):

    '''
    Input: Asset symbol - 'btc', 'eth', 'ltc', 'dot', 'xmr', 'xdg', 'xlm', 'xrp', 'zec', 'nano', 'trx', 'bch', 'xtz', 'ada','oxt'
    Returns:  Dataframe of technial indicator signals

    NOTICE: When analyzing on the daily timeframe or greater, VWAP will not apply as it is ONLY an intraday indicator.
    '''

    # create a dataframe to house the technical trading signals
    asset_df = kraken_data(asset)

    technical_signals = pd.DataFrame({
        'close': asset_df.Close,
        'ewma_x': ewma_crossover(asset_df).signal,
        'macd': macd(asset_df).signal,
        'bollinger': b_band(asset_df).signal,
        'rsi': rsi(asset_df).signal,
        'psar': psar(asset_df).signal,
        'vwap': vwap(asset_df).signal 
    })

    # since VWAP won't work on daily time intervals and greater, we need to check the interval to see if we should include vwap as a column or not
    daily_seconds = 86400
    delta_seconds = timedelta.total_seconds(technical_signals.index[1] - technical_signals.index[0])
    interval = delta_seconds / daily_seconds

    if interval >= 1:
        # if the interval is greater than or equal to 1, then do not include VWAP
        technical_signals = technical_signals.drop(columns='vwap')

    # sum the various technical signals together to return a trade 'grade' or signal
    technical_signals['signal'] = technical_signals.drop(columns=['close']).sum(axis='columns')

    return technical_signals

technical_signals = technical_indicator_signal('xdg')


In [3]:
# trade_cutoff = 1

# btc_df['trade'] = np.where(btc_df.signal >= trade_cutoff, 1, 0)

btc_df.dropna(inplace=True)
btc_df.drop(columns=['signal'], inplace=True)
btc_df['returns'] = btc_df.close.pct_change()
btc_df.tail()

,close,ewma_x,macd,bollinger,rsi,psar,returns
Date,,,,,,,
2021-01-04 18:00:00,0.010000,1,1.0,-1,-1.0,1,0.021200
2021-01-05 18:00:00,0.010458,1,1.0,-1,-1.0,1,0.045800
2021-01-06 18:00:00,0.009768,1,1.0,-1,-1.0,1,-0.066016
2021-01-07 18:00:00,0.009794,1,1.0,-1,-1.0,-1,0.002703
2021-01-08 18:00:00,0.010290,1,1.0,-1,-1.0,-1,0.050643


In [4]:
# create a random forest model for signal weights
x_var_list = ['ewma_x', 'macd', 'bollinger', 'rsi', 'psar']

In [5]:
days_to_forecast = 1

shifted_df = btc_df
shifted_df[x_var_list] = shifted_df[x_var_list].shift(days_to_forecast)
shifted_df.tail()

,close,ewma_x,macd,bollinger,rsi,psar,returns
Date,,,,,,,
2021-01-04 18:00:00,0.010000,1.0,1.0,-1.0,-1.0,1.0,0.021200
2021-01-05 18:00:00,0.010458,1.0,1.0,-1.0,-1.0,1.0,0.045800
2021-01-06 18:00:00,0.009768,1.0,1.0,-1.0,-1.0,1.0,-0.066016
2021-01-07 18:00:00,0.009794,1.0,1.0,-1.0,-1.0,1.0,0.002703
2021-01-08 18:00:00,0.010290,1.0,1.0,-1.0,-1.0,-1.0,0.050643


In [6]:
shifted_df.dropna(subset=x_var_list, inplace=True)
shifted_df.dropna(subset=['returns'], inplace=True)
shifted_df = shifted_df.replace([np.inf, -np.inf], np.nan)
shifted_df.head()

,close,ewma_x,macd,bollinger,rsi,psar,returns
Date,,,,,,,
2019-12-20 18:00:00,0.002100,-1.0,-1.0,1.0,1.0,-1.0,0.039707
2019-12-21 18:00:00,0.002085,-1.0,-1.0,1.0,1.0,-1.0,-0.007143
2019-12-22 18:00:00,0.002232,-1.0,-1.0,1.0,1.0,-1.0,0.070360
2019-12-23 18:00:00,0.002008,1.0,0.0,1.0,1.0,-1.0,-0.100237
2019-12-24 18:00:00,0.002020,-1.0,-1.0,1.0,1.0,-1.0,0.006125


In [7]:
# make returns binary -- above 0 == 1, equal to or below 0 == 0
shifted_df['positive_return'] = np.where(shifted_df.returns > 0, 1, 0)
shifted_df

,close,ewma_x,macd,bollinger,rsi,psar,returns,positive_return
Date,,,,,,,,
2019-12-20 18:00:00,0.002100,-1.0,-1.0,1.0,1.0,-1.0,0.039707,1
2019-12-21 18:00:00,0.002085,-1.0,-1.0,1.0,1.0,-1.0,-0.007143,0
2019-12-22 18:00:00,0.002232,-1.0,-1.0,1.0,1.0,-1.0,0.070360,1
2019-12-23 18:00:00,0.002008,1.0,0.0,1.0,1.0,-1.0,-0.100237,0
2019-12-24 18:00:00,0.002020,-1.0,-1.0,1.0,1.0,-1.0,0.006125,1
...,...,...,...,...,...,...,...,...
2021-01-04 18:00:00,0.010000,1.0,1.0,-1.0,-1.0,1.0,0.021200,1
2021-01-05 18:00:00,0.010458,1.0,1.0,-1.0,-1.0,1.0,0.045800,1
2021-01-06 18:00:00,0.009768,1.0,1.0,-1.0,-1.0,1.0,-0.066016,0


In [8]:
training_per = 0.6

training_start = shifted_df.index.min().strftime(format= '%Y-%m-%d')
training_end = shifted_df.iloc[:int(len(shifted_df) * training_per)].index.max().strftime(format= '%Y-%m-%d')

testing_start = shifted_df[training_end:].iloc[1:].index.min().strftime(format= '%Y-%m-%d')
testing_end = shifted_df[testing_start:].index.max().strftime(format= '%Y-%m-%d')

print(training_start)
print(training_end)
print(testing_start)
print(testing_end)

2019-12-20
2020-08-06
2020-08-07
2021-01-08


In [9]:
# create the test/train split
X_train = shifted_df[x_var_list][training_start:training_end]
y_train = shifted_df.positive_return[training_start:training_end]

X_test = shifted_df[x_var_list][testing_start:]
y_test = shifted_df.positive_return[testing_start:]

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [11]:
model = RandomForestClassifier(n_estimators=100000, max_depth=10, random_state=0)

model.fit(X_train, y_train)

predictions = model.predict(X_test)

In [12]:
prediction = model.predict(shifted_df[x_var_list])

results = shifted_df[['returns', 'positive_return']]
results['predict'] = prediction

results['predict'].replace(0 , -1, inplace=True)

results

,returns,positive_return,predict
Date,,,
2019-12-20 18:00:00,0.039707,1,-1
2019-12-21 18:00:00,-0.007143,0,-1
2019-12-22 18:00:00,0.070360,1,-1
2019-12-23 18:00:00,-0.100237,0,-1
2019-12-24 18:00:00,0.006125,1,-1
...,...,...,...
2021-01-04 18:00:00,0.021200,1,1
2021-01-05 18:00:00,0.045800,1,1
2021-01-06 18:00:00,-0.066016,0,1


In [23]:
capital = shifted_df.close[0]

cum_return = capital * (1 + (results.returns * results.predict)).cumprod()
print(f'Starting Capital = ${capital}')
print(f'Cumulative Portfolio = ${format(float(cum_return[-1]), "0.4f")}')
(cum_return.hvplot() * shifted_df.close.hvplot()).opts(show_legend=False)



Starting Capital = $0.0021
Cumulative Portfolio = $0.0252


:Overlay
   .Curve.A_0   :Curve   [Date]   (0)
   .Curve.Close :Curve   [Date]   (close)